# Extracting and Visualizing Stock Data

In [1]:
import warnings
import requests
import pandas as pd
import yfinance as yf
import plotly.graph_objects as go
import plotly.io as pio
from bs4 import BeautifulSoup
from pandas.core.frame import DataFrame
from plotly.subplots import make_subplots

In [2]:
# Ignore warnings
warnings.filterwarnings('ignore', category=FutureWarning)

## Define Graphing Function

In [46]:
def make_graph(stock_data: DataFrame, revenue_data: DataFrame, stock: str):
    '''Plot the given `stock` from the `stock_data`'''
    # Define a plot with two subplots and shared x-axis
    fig = make_subplots(
        rows=2, cols=1, shared_xaxes=True,
        subplot_titles=('Historical Share Price', 'Historical Revenue'),
        vertical_spacing=.3
    )
    # Use data before certain dates
    stock_data_specific = stock_data[stock_data.Date <= '2021-06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    # Add traces
    fig.add_trace(
        go.Scatter(
            x=pd.to_datetime(stock_data_specific.Date),
            y=stock_data_specific.Close.astype('float'), name='Share Price',
        ),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(
            x=pd.to_datetime(revenue_data_specific.Date),
            y=revenue_data_specific.Revenue.astype('float'), name='Revenue',
        ),
        row=2, col=1
    )
    # Update layout
    fig.update_xaxes(title_text='Date', row=1, col=1)
    fig.update_xaxes(title_text='Date', row=2, col=1)
    fig.update_yaxes(title_text='Price ($US)', row=1, col=1)
    fig.update_yaxes(title_text='Revenue ($US Millions)', row=2, col=1)
    fig.update_layout(
        showlegend=False,
        height=900,
        width=1100,
        title=stock,
        xaxis_rangeslider_visible=True
    )
    fig.show()

## Q1

In [22]:
# Extract Tesla stock price
tesla = yf.Ticker('TSLA')
tesla_data = tesla.history(period='max')
# Reset index inplace
tesla_data.reset_index(inplace=True)

In [23]:
# Showing first 5 rows
tesla_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
1,2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2,2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
3,2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
4,2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0


## Q2

In [24]:
# Web scraping TESLA data from the url
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm'
# Saving the response as a text object
html_data = requests.get(url).text
# Parsing the response
soup = BeautifulSoup(html_data, features='html.parser')
# Extracting the revenue table into a dataframe
tesla_revenue = pd.DataFrame(columns=['Date', 'Revenue'])
for row in soup.find('tbody').find_all('tr'):
    col = row.find_all('td')
    date = col[0].text
    rev = col[1].text
    tesla_revenue = pd.concat(
        objs=[
            tesla_revenue,
            pd.DataFrame({'Date': [date], 'Revenue': [rev]})
        ]
    )
# Reset the index
tesla_revenue.reset_index(drop=True, inplace=True)
# Remove comma and dollar signs from the revenue values
tesla_revenue.Revenue = tesla_revenue.Revenue.str.replace(',', '')
tesla_revenue.Revenue = tesla_revenue.Revenue.str.replace('$', '')
# Display the last 5 rows
tesla_revenue.tail()

,Date,Revenue
8,2013,2013
9,2012,413
10,2011,204
11,2010,117
12,2009,112


## Q3

In [25]:
# GameStop data
gme = yf.Ticker('GME')
gme_data = gme.history(period='max')
gme_data.reset_index(inplace=True)
gme_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2002-02-13 00:00:00-05:00,1.620129,1.693350,1.603296,1.691667,76216000,0.0,0.0
1,2002-02-14 00:00:00-05:00,1.712707,1.716073,1.670626,1.683250,11021600,0.0,0.0
2,2002-02-15 00:00:00-05:00,1.683250,1.687458,1.658001,1.674834,8389600,0.0,0.0
3,2002-02-19 00:00:00-05:00,1.666418,1.666418,1.578047,1.607504,7410400,0.0,0.0
4,2002-02-20 00:00:00-05:00,1.615920,1.662210,1.603296,1.662210,6892800,0.0,0.0


## Q4

In [26]:
url2 = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html'
html_data_2 = requests.get(url2).text
soup2 = BeautifulSoup(html_data_2, features='html.parser')
# Extracting the revenue table into a dataframe
gme_revenue = pd.DataFrame(columns=['Date', 'Revenue'])
for row in soup2.find('tbody').find_all('tr'):
    col = row.find_all('td')
    date = col[0].text
    rev = col[1].text
    gme_revenue = pd.concat(
        objs=[
            gme_revenue,
            pd.DataFrame({'Date': [date], 'Revenue': [rev]})
        ]
    )
# Reset the index
gme_revenue.reset_index(drop=True, inplace=True)
# Remove comma and dollar signs from the revenue values
gme_revenue.Revenue = gme_revenue.Revenue.str.replace(',', '')
gme_revenue.Revenue = gme_revenue.Revenue.str.replace('$', '')
gme_revenue.tail()

,Date,Revenue
11,2009,8806
12,2008,7094
13,2007,5319
14,2006,3092
15,2005,1843


## Q5

In [28]:
# Plotting
make_graph(tesla_data, tesla_revenue, 'TESLA')

## Q6

In [47]:
# Plotting
make_graph(gme_data, gme_revenue, 'GameStop')